In [1]:
import os
os.environ['TRKXINPUTDIR'] = '/fs/ess/PLS0151/trackPred/' # better change to your copy of the dataset.
os.environ['TRKXOUTPUTDIR'] = '../run200' # change to your own directory

In [2]:
import torch; 
print(torch.version.cuda)

11.0


In [3]:
# system import
import pkg_resources
import yaml
import pprint
import random
random.seed(1234)
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
#import psutil 
# %matplotlib widget

# 3rd party
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from trackml.dataset import load_event
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

import torch.utils.data as data_utils
from torch.utils.data import Dataset, DataLoader

from trackml.dataset import load_event
from exatrkx.src.processing.utils.detector_utils import load_detector
from exatrkx.src.processing.utils.cell_utils import get_one_event

# for embedding
from exatrkx import LayerlessEmbedding
#from trained_models_v0.Embedding.model.Models.layerless_embedding import LayerlessEmbedding
from exatrkx.src import utils_torch
#import pynndescent
# for filtering
#from exatrkx import VanillaFilter
from trained_models_v0.Filter.model.Models.vanilla_filter import VanillaFilter
from trained_models_v0.GNN.model.Models.agnn import ResAGNN
#from exatrkx import ResAGNN

# for GNN
import tensorflow as tf
from graph_nets import utils_tf
from exatrkx import SegmentClassifier
import sonnet as snt

# for labeling
from exatrkx.scripts.tracks_from_gnn import prepare as prepare_labeling
from exatrkx.scripts.tracks_from_gnn import clustering as dbscan_clustering
#from exatrkx.scripts.tracks_from_gnn import hclustering as hdbscan_clustering
import networkx as nx
import cudf
import cugraph
# track efficiency
from trackml.score import _analyze_tracks
from exatrkx.scripts.eval_reco_trkx import make_cmp_plot, pt_configs, eta_configs
from functools import partial

True 11.0


ImportError: cannot import name 'random_edge_slice_v2' from 'trained_models_v0.GNN.model.utils' (/users/PLS0129/ysu0053/torchScript/trained_models_v0/GNN/model/utils.py)

In [ ]:
import exatrkx
print(exatrkx.__file__)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

### Setup some hyperparameters and event

In [ ]:
embed_ckpt_dir = '/users/PLS0129/ysu0053/torchScript/trained_models_v0/Embedding/trained/checkpoints/last.ckpt'
filter_ckpt_dir = '/users/PLS0129/ysu0053/torchScript/trained_models_v0/Filter/trained/checkpoints/last.ckpt'
gnn_ckpt_dir = '/users/PLS0129/ysu0053/torchScript/trained_models_v0/GNN/trained/checkpoints/last.ckpt'
plots_dir = '../run200' # needs to change...
ckpt_idx = -1 # which GNN checkpoint to load
dbscan_epsilon, dbscan_minsamples = 0.25, 2 # hyperparameters for DBScan
min_hits = 5 # minimum number of hits associated with a particle to define "reconstructable particles"
frac_reco_matched, frac_truth_matched = 0.5, 0.5 # parameters for track matching

In [ ]:
detector_path='/fs/ess/PLS0151/trackPred/detectors.csv'
input_path = "/fs/ess/PLS0151/trackPred/train_100_events/"
detector_orig, detector_proc = load_detector(detector_path)

In [ ]:
def prepare_one_event(evtid):
    event_file = os.path.join(input_path, 'event{:09}'.format(evtid))
    hits, particles, truth = load_event(event_file, parts=['hits', 'particles', 'truth'])

    r = np.sqrt(hits.x**2 + hits.y**2)
    phi = np.arctan2(hits.y, hits.x)
    hits = hits.assign(r=r, phi=phi)
    hits = hits.merge(truth, on='hit_id')
    hits = hits[hits['particle_id'] != 0]
    hits = hits.merge(particles, on='particle_id', how='left')
    hits = hits[hits.nhits >= min_hits]
    particles = particles[particles.nhits >= min_hits]

    angles = get_one_event(event_file, detector_orig, detector_proc)
    hits = hits.merge(angles, on='hit_id')

    cell_features = ['cell_count', 'cell_val', 'leta', 'lphi', 'lx', 'ly', 'lz', 'geta', 'gphi']
    feature_scale = np.array([1000, np.pi, 1000])
    hid = hits['hit_id'].to_numpy()
    x = hits[['r', 'phi', 'z']].to_numpy() / feature_scale
    cell_data = hits[cell_features].to_numpy()
    return hid, x, cell_data

In [ ]:
min_hits = 5
evtid =1000
hid, x, cell_data = prepare_one_event(evtid)
data = Data(hid=torch.from_numpy(hid),
            x=torch.from_numpy(x).float(),
            cell_data=torch.from_numpy(cell_data).float(),
            pin_memory=True).to(device)

### Evaluating Embedding

In [ ]:
e_ckpt = torch.load(embed_ckpt_dir, map_location='cpu')
e_config = e_ckpt['hyper_parameters']
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(e_config)

In [ ]:
e_config = e_ckpt['hyper_parameters']
e_config['clustering'] = 'build_edges'
e_config['knn_val'] = 500
e_config['r_val'] = 1.6
e_config['in_channels'] = 3
pp.pprint(e_config)

Load the checkpoint and put the model in the evaluation state.

In [ ]:
e_model = LayerlessEmbedding(e_config).to(device)
e_model.load_state_dict(e_ckpt["state_dict"])

In [ ]:
e_model.eval()
#e_model.half()

In [ ]:
with torch.jit.optimized_execution(True):
    e_script = e_model.to_torchscript()
#fe_script = torch.jit.freeze(e_script)
# Save to file
torch.jit.save(e_script, 'embed.pt')

In [ ]:
e_script.code
#fe_script.hparams['r_val']

In [ ]:
e_script = torch.jit.load('embed.pt')

Map each hit to the embedding space, return the embeded parameters for each hit

In [ ]:
torch.cat([data.cell_data, data.x], axis=-1).size()

In [ ]:
e_input = torch.full((5, 3), 1.0).to(device)
e_input = data.x
e_df = pd.read_csv("data/in_e.csv", header=None, dtype=np.float32)
e= e_df.values
e_input = torch.tensor(e, dtype=torch.float32).to(device)

In [ ]:
%%time
with torch.no_grad():
    #with torch.cuda.amp.autocast():
    #spatial = e_model(e_input)
    spatial = e_script(e_input)
spatial

### From embeddeding space form doublets

`r_val = 1.7` and `knn_val = 500` are the hyperparameters to be studied.

* `r_val` defines the radius of the clustering method
* `knn_val` defines the number of maximum neighbors in the embedding space

In [ ]:
%%time
#e_spatial = utils_torch.build_edges_baseline(spatial.to(device), e_model.hparams['r_val'], e_model.hparams['knn_val'])
e_spatial = utils_torch.build_edges(spatial.to(device),1.6 ,500) #e_model.hparams['r_val']
e_spatial

Removing edges that point from outer region to inner region, which almost removes half of edges.

In [ ]:
%%time
R_dist = torch.sqrt(data.x[:,0]**2 + data.x[:,2]**2) # distance away from origin...
e_spatial = e_spatial[:, (R_dist[e_spatial[0]] <= R_dist[e_spatial[1]])]
e_spatial

### Filtering


In [ ]:
f_ckpt = torch.load(filter_ckpt_dir, map_location='cpu')
f_config = f_ckpt['hyper_parameters']
pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(f_config)

In [ ]:
f_config['train_split'] = [0, 0, 1]
f_config['filter_cut'] = 0.2

In [ ]:
f_model = VanillaFilter(f_config).to(device)
# f_model = f_model.load_from_checkpoint(filter_ckpt_dir, hparams=f_config)
f_model.load_state_dict(f_ckpt['state_dict'])

In [ ]:
f_model.eval()
#f_model.half()

In [ ]:
with torch.jit.optimized_execution(True):
    f_script = f_model.to_torchscript()
#ff_script = torch.jit.freeze(f_script)
# Save to file
torch.jit.save(f_script, 'filter.pt')
#f_script.graph

In [ ]:
#%%time
#emb = None # embedding information was not used in the filtering stage.
#output = f_model(torch.cat([data.cell_data, data.x], axis=-1), e_spatial, emb).squeeze()

In [ ]:
%%time
emb = None # embedding information was not used in the filtering stage.
chunks = 8
output_list = []
for j in range(chunks):
    subset_ind = torch.chunk(torch.arange(e_spatial.shape[1]), chunks)[j]
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            #output = f_script(torch.cat([data.cell_data, data.x], axis=-1).to(device), e_spatial[:, subset_ind]).squeeze()
            output = f_script(e_input.to(device), e_spatial[:, subset_ind]).squeeze()
            #output = f_model(torch.cat([data.cell_data, data.x], axis=-1).to(device), e_spatial[:, subset_ind]).squeeze() 
    output_list.append(output)
output = torch.cat(output_list)

In [ ]:
output = torch.sigmoid(output)
print(output)

In [ ]:
output.shape, e_spatial.shape

In [ ]:
# this plot may need some time to load...
plt.hist(output.detach().to('cpu').numpy(), bins=100, histtype='step', lw=2)
plt.show()
plt.yscale('log')

The filtering network assigns a score to each edge. In the end, edges with socres > `filter_cut` are selected to construct graphs.

In [ ]:
edge_list = e_spatial[:, output.to('cpu') > f_model.hparams['filter_cut']]

In [ ]:
edge_list.shape

### Form a graph
Now moving TensorFlow for GNN inference.

In [ ]:
n_nodes = data.x.shape[0]
n_edges = edge_list.shape[1]
#nodes = data.x.numpy().astype(np.float32)
nodes = data.x #.cpu().numpy().astype(np.float32)
#edge_list = edge_list.cpu().numpy().astype(np.int32)
#edges = np.zeros((n_edges, 1), dtype=np.float32)
#senders = edge_list[0].cpu()
#receivers = edge_list[1].cpu()

In [ ]:
nodes[0]

In [ ]:
edge_list

In [ ]:
#input_datadict = {
#    "n_node": n_nodes,
#    "n_edge": n_edges,
#    "nodes": nodes,
#    "edges": edges,
#    "senders": senders,
#    "receivers": receivers,
#    "globals": np.array([n_nodes], dtype=np.float32)
#}

In [ ]:
#input_graph = utils_tf.data_dicts_to_graphs_tuple([input_datadict])

### Apply GNN

In [ ]:
#num_processing_steps_tr = 8
#optimizer = snt.optimizers.Adam(0.001)
#model = SegmentClassifier()

#output_dir = gnn_ckpt_dir
#checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
#ckpt_manager = tf.train.CheckpointManager(checkpoint, directory=output_dir, max_to_keep=10)
#status = checkpoint.restore(ckpt_manager.checkpoints[ckpt_idx]).expect_partial()
#print("Loaded {} checkpoint from {}".format(ckpt_idx, output_dir))

In [ ]:
checkpoint = torch.load("last.ckpt", map_location=device)
hparams = checkpoint["hyper_parameters"]
state_dict = checkpoint["state_dict"]

g_model = ResAGNN(hparams).to(device)
g_model.load_state_dict(state_dict)
g_model.eval()
#g_model.half()

In [ ]:
input_data = (data.x, edge_list)
g_script = g_model.to_torchscript(file_path="gnn_script.pt",example_inputs=input_data)
#g_script = torch.jit.script(model, input_data)

In [ ]:
with torch.jit.optimized_execution(True):
    g_script = g_model.to_torchscript()
#g_script = torch.jit.freeze(g_script)
# Save to file
torch.jit.save(g_script, 'gnn.pt')

In [ ]:
#%time
#outputs_gnn = model(nodes, edge_list)
#output_graph = outputs_gnn[-1]

#with torch.no_grad():
#   with torch.cuda.amp.autocast():
#       gnnoutput = g_model(data.x,edge_list)
#nnoutput = torch.sigmoid(gnnoutput).cpu().numpy()

In [ ]:
%%time
with torch.no_grad():
    with torch.cuda.amp.autocast():
        gnnoutput = g_script(data.x, edge_list)
        #gnnoutput = g_model(data.x, edge_list)
gnnoutput = torch.sigmoid(gnnoutput).cpu().numpy()

### Track labeling

In [ ]:
#input_matrix = prepare_labeling(tf.squeeze(output_graph.edges).cpu().numpy(), senders, receivers, n_nodes)

In [ ]:
from importlib import reload 
import exatrkx.scripts.tracks_from_gnn
reload(exatrkx.scripts.tracks_from_gnn)
from exatrkx.scripts.tracks_from_gnn import clustering as dbscan_clustering

In [ ]:
%%time
#predict_tracks = dbscan_clustering(data.hid.cpu(), input_matrix, dbscan_epsilon, dbscan_minsamples)
hids = data.hid.cpu().numpy()
cut_edges = hids[edge_list.cpu().numpy()][:,gnnoutput > 0.75]
cut_df = cudf.DataFrame(cut_edges.T)
G=cugraph.Graph()
G.from_cudf_edgelist(cut_df,source=0, destination=1, edge_attr=None)
labels = cugraph.components.connectivity.weakly_connected_components(G)
predict_tracks_df = labels.to_pandas()
predict_tracks_df.columns = ["track_id","hit_id"]

### Track Efficiency

In [ ]:
event_file = os.path.join(input_path, 'event{:09}'.format(evtid))
hits, particles, truth = load_event(event_file, parts=['hits', 'particles', 'truth'])
hits = hits.merge(truth, on='hit_id', how='left')
hits = hits[hits.particle_id > 0] # remove noise hits
hits = hits.merge(particles, on='particle_id', how='left')
hits = hits[hits.nhits >= min_hits]
particles = particles[particles.nhits >= min_hits]
par_pt = np.sqrt(particles.px**2 + particles.py**2)
momentum = np.sqrt(particles.px**2 + particles.py**2 + particles.pz**2)
ptheta = np.arccos(particles.pz/momentum)
peta = -np.log(np.tan(0.5*ptheta))

In [ ]:
tracks = _analyze_tracks(hits, predict_tracks_df)

In [ ]:
purity_rec = np.true_divide(tracks['major_nhits'], tracks['nhits'])
purity_maj = np.true_divide(tracks['major_nhits'], tracks['major_particle_nhits'])
good_track = (frac_reco_matched < purity_rec) & (frac_truth_matched < purity_maj)

matched_pids = tracks[good_track].major_particle_id.values
score = tracks['major_weight'][good_track].sum()

n_recotable_trkx = particles.shape[0]
n_reco_trkx = tracks.shape[0]
n_good_recos = np.sum(good_track)
matched_idx = particles.particle_id.isin(matched_pids).values

In [ ]:
print("Processed {} events from {}".format(evtid, input_path))
print("Reconstructable tracks:         {}".format(n_recotable_trkx))
print("Reconstructed tracks:           {}".format(n_reco_trkx))
print("Reconstructable tracks Matched: {}".format(n_good_recos))
print("Tracking efficiency:            {:.4f}".format(n_good_recos/n_recotable_trkx))
print("Tracking purity:               {:.4f}".format(n_good_recos/n_reco_trkx))

In [ ]:
make_cmp_plot_fn = partial(make_cmp_plot, xlegend="Matched", ylegend="Reconstructable",
                    ylabel="Events", ratio_label='Track efficiency')

In [ ]:
make_cmp_plot_fn(par_pt[matched_idx], par_pt,
                 configs=pt_configs,
                 xlabel="pT [GeV]",
                 outname=os.path.join(plots_dir, "{}_pt".format(evtid)))

In [ ]:
make_cmp_plot_fn(peta[matched_idx], peta,
                 configs=eta_configs,
                 xlabel=r"$\eta$",
                 outname=os.path.join(plots_dir, "{}_eta".format(evtid)))

In [ ]:
def prepare_one_event(evtid):
    event_file = os.path.join(input_path, 'event{:09}'.format(evtid))
    hits, particles, truth = load_event(event_file, parts=['hits', 'particles', 'truth'])

    r = np.sqrt(hits.x**2 + hits.y**2)
    phi = np.arctan2(hits.y, hits.x)
    hits = hits.assign(r=r, phi=phi)
    hits = hits.merge(truth, on='hit_id')
    hits = hits[hits['particle_id'] != 0]
    hits = hits.merge(particles, on='particle_id', how='left')
    hits = hits[hits.nhits >= min_hits]
    particles = particles[particles.nhits >= min_hits]

    angles = get_one_event(event_file, detector_orig, detector_proc)
    hits = hits.merge(angles, on='hit_id')

    cell_features = ['cell_count', 'cell_val', 'leta', 'lphi', 'lx', 'ly', 'lz', 'geta', 'gphi']
    feature_scale = np.array([1000, np.pi, 1000])
    hid = hits['hit_id'].to_numpy()
    x = hits[['r', 'phi', 'z']].to_numpy() / feature_scale
    cell_data = hits[cell_features].to_numpy()
    return hid, x, cell_data

In [ ]:
evtid =1000
hid, x, cell_data = prepare_one_event(evtid)
data = Data(hid=torch.from_numpy(hid),
            x=torch.from_numpy(x).float(),
            cell_data=torch.from_numpy(cell_data).float(),
            pin_memory=True).to(device)

In [ ]:
e_script.code

In [ ]:
e_script._parameters.values()